In [15]:
from glob import glob
import pandas as pd
import numpy as np
import xarray as xr
import holoviews as hv
import hvplot.pandas, hvplot.xarray
from holoviews import opts
import dask
import dask.bag as db
import intake

hvplot.extension('matplotlib')

from lib.preamble import *

import warnings
warnings.filterwarnings("ignore")
# %matplotlib widget

In [16]:
import h5py
paths = glob("../data/2023-12-01/d*.h5")
paths += glob("../data/2023-12-04*/*.h5")
paths += glob("../data/2023-12-05*/*.h5")

def custom_import(p):
	try:
		with h5py.File(p, "r") as f:
			wavelength = np.array(f['x'])[0]
			angle = np.array(f['y'])
			counts = np.array(f['z'])
	except:
		print(f"Error importing {p}")
		return None
	
	ar = xr.DataArray(
		counts,
		coords={
			'angle': angle,
			'wavelength': wavelength
		}
	)

	ar.attrs = {
		**ar.attrs,
		"path": p.split("data/")[-1],
		"name": ".".join(p.split("data/")[-1].split(".")[:-1]),
	}
	ar.name = ar.attrs["name"]
	
	return ar

paths.sort()
data = [custom_import(p) for p in paths]
data = [d for d in data if d is not None]

Error importing ../data/2023-12-05_LO_MG_NiPS3/d004_10K_647nm_rotPolDet_26degPolExc_flake04.hd5.h5


In [17]:
# import backgrounds
paths = glob("../data/2023-12-01/d*.asc")
paths += glob("../data/2023-12-05*/d*.asc")
paths = [p for p in paths if "bkg" in p]
paths.sort()

import csv

def _read_csv(path):
	rows = []
	with open(path) as f:
		r = csv.reader(f, delimiter="\t")
		for row in r:
			if len(row) == 0:
				return rows
			rows.append(row)
	return rows

def custom_import(p):
	d = np.array(_read_csv(p))[:, :-1]
	# convert strings to floats
	d = d.astype(float)

	ar = xr.DataArray(
		d[:, 1],
		coords={
			'wavelength': d[:, 0]
		}
	)

	ar.attrs = {
		**ar.attrs,
		"path": p.split("data/")[-1],
		"name": p.split("/")[-1].split(".")[0],
	}
	ar.name = ar.attrs["name"]

	return ar

bkg = [custom_import(p) for p in paths]

bkg[1] = bkg[1] / 10
bkg[2] = bkg[2] / 3

def subtract_background(sel, b=bkg[-1]):
	d =  sel - b.interp(wavelength=sel.wavelength)
	d.attrs = sel.attrs
	d.name = sel.name
	d.attrs["background_subtracted"] = True
	return d

# Quick Preview

In [18]:
{i:d.name for i, d in enumerate(data)}

{0: '2023-12-01/d001_pol_30K_0T_1mW_647nm_HorizPolDetektor_rotatingLaserPolarisation.hd5',
 1: '2023-12-01/d002_pol_30K_0T_561nm_HorizPolDetektor_rotatingLaserPolarisation.hd5',
 2: '2023-12-01/d003_pol_30K_0T_561nm_HorizPolDetektor_rotatingLaserPolarisation.hd5',
 3: '2023-12-01/d004_pol_30K_0T_647nm_rotatingPolDetektor_30degLaserPolarisation.hd5',
 4: '2023-12-01/d005_pol_30K_0T_647nm_rotatingPolDetektor_72degLaserPolarisation.hd5',
 5: '2023-12-04_LO_MG_NiPS3/d001_pol_30K_0T_647nm_circPolDet_rotatingPolExc',
 6: '2023-12-04_LO_MG_NiPS3/d002_pol_30K_0T_647nm_0degPolDet_rotatingPolExc',
 7: '2023-12-04_LO_MG_NiPS3/d002_pol_30K_0T_647nm_11.25degPolDet_rotatingPolExc',
 8: '2023-12-04_LO_MG_NiPS3/d002_pol_30K_0T_647nm_22.5degPolDet_rotatingPolExc',
 9: '2023-12-04_LO_MG_NiPS3/d002_pol_30K_0T_647nm_33.75degPolDet_rotatingPolExc',
 10: '2023-12-04_LO_MG_NiPS3/d002_pol_30K_0T_647nm_45degPolDet_rotatingPolExc',
 11: '2023-12-04_LO_MG_NiPS3/d003_pol_30K_0T_647nm_rotPolDet_27degPolExc_flake02

In [19]:
# quick preview of measurement

sel = data[3].copy()
sel = data[-1].copy()
# sel = data[11].copy()
name = sel.name
sel = subtract_background(sel)

wavlength_sel = [840.07, 832.5, 850]
angle_sel = np.linspace(0, sel.angle.max()-15, 1)

# sel = sel /sel.sel(wavelength=slice(830, 835)).mean("wavelength")
# sel = sel / sel.max("angle")

# sel = sel.interp(angle=np.linspace(sel.angle.min(), sel.angle.max(), 200))		# interpolate angle
sel = sel.rolling(wavelength=3, center=True).mean()								# smooth along wavelength

plt.figure(figsize=FIGSIZE_WIDE)
ax0 = plt.subplot(1,2,1)
sel.name = ""
sel.plot(cmap="magma")
plt.title(name, fontsize=FONTSIZE_TINY)

for wl in wavlength_sel:
	plt.axvline(wl, color="white", linestyle="--", alpha=.5)

for a in angle_sel:
	# plt.axhline(a, color="white", linestyle="--", alpha=.5)
	plt.plot(
		sel.wavelength, 
		a + 20*(sel.sel(angle=a, method="nearest") / sel.max()),
		color="white"
	)

# sel.idxmax('wavelength').plot(color="w", y="angle")
# plt.show()


ax = plt.subplot(1,2,2, projection="polar")
# fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
sel = sel.assign_coords(angle=sel.angle * np.pi/180)
sel = sel.assign_coords(angle=sel.angle * 2)

def fit(x, *p): return p[0] * np.cos(x + p[1])**2 + p[2]

for i, wl in enumerate(wavlength_sel):
	sel.sel(
		wavelength=wl,
		method="nearest"
	).plot.scatter(
		label=f"{wl:.0f} nm",
		s=30
	)
	popt, pcov = sp.optimize.curve_fit(fit, 
		sel.angle, sel.sel(wavelength=wl, method="nearest"), 
		p0=[1, 0, 0]
	)
	# print(f"best angle for {wl:.0f} nm: {np.abs(popt[1])%(np.pi/2) * 180/np.pi:.2f}° +- {np.sqrt(pcov[1,1]) * 180/np.pi:.2f}°")
	angle = np.linspace(0, 2*np.pi, 100)
	plt.plot(angle, fit(angle, *popt),
		label=f"{np.abs(popt[1])%(np.pi/2) * 180/np.pi:.1f}±{np.sqrt(pcov[1,1]) * 180/np.pi:.1f}°",
		linestyle="--" if i != 0 else "-"
	)

plt.legend(bbox_to_anchor=(1.1, 1))
# plt.title(name, fontsize=FONTSIZE_TINY)
plt.title("")
plt.ylabel("")
plt.xlabel("")
ax.set_rlabel_position(-90)

plt.show()

In [20]:
plt.figure()
plt.plot(sel.wavelength, sel.sel(angle=0, method="nearest"), label="0")
plt.plot(sel.wavelength, sel.sel(angle=45/2, method="nearest"), label="45/2")
plt.plot(sel.wavelength, sel.sel(angle=45, method="nearest"), label="45")
plt.legend()
plt.show()

In [21]:
sel0 = sel.sel(
		wavelength=840,
		method="nearest"
	)

## Preview of Excitation and Luminescence Polarisation Dependence

In [22]:
# systematic excitation and detection polarisation dependence
sel = data[3:5]
sel = [d.copy() for d in sel]
sel = [subtract_background(d) for d in sel]
sel = [d.rolling(wavelength=5).mean() for d in sel]								# smooth along wavelength

print(sel[0].name)

def norm(d):
	return d.max("wavelength")

def fit(x, *p): return p[0] * np.cos(x + p[1])**2 + p[2]

plt.figure()
popt = []
popt_unc = []
ax = plt.subplot(1,1,1, projection="polar")
for d in sel:
	d = d.assign_coords(angle=d.angle * np.pi/180)
	d = d.assign_coords(angle=d.angle * 2)
	norm(
		d
	).plot.scatter(label=d.name, s=30)
	opot, pcov = sp.optimize.curve_fit(fit, d.angle, norm(d), p0=[1, 0, 0])
	angle = np.linspace(0, 2*np.pi, 100)
	plt.plot(angle, fit(angle, *opot))
	popt.append(opot)
	popt_unc.append(np.sqrt(np.diag(pcov)))
plt.legend(bbox_to_anchor=(1.1, 1))
plt.xlabel("excite pol. angle")
plt.title("")
plt.ylabel("")
ax.set_rlabel_position(-90)
plt.show()

2023-12-01/d004_pol_30K_0T_647nm_rotatingPolDetektor_30degLaserPolarisation.hd5


In [23]:
# Preview of Excitation and Luminescence Polarisation Dependence

def linear_dichroism(d):
	d_bkg = subtract_background(d)
	dichroism = (d_bkg.max("angle") - d_bkg.min("angle")) / (d_bkg.max("angle") + d_bkg.min("angle"))	
	return dichroism.rolling_exp(wavelength=10).mean()

def extract_LaserPolarisation(p):
	return p.split("Laser")[0].split("_")[-1]

def plot(d):
	linear_dichroism(d).plot(label=extract_LaserPolarisation(d.name))

plt.figure()
plot(data[3])
plot(data[4])
plt.ylabel("deg. of pol. in PL")
plt.legend()
# plt.savefig("../figures/2023-12-01_linear_dichroism.png")
plt.show()

sel = data[3:5]
print([d.name for d in sel])
sel = [subtract_background(d) for d in sel]
sel = [d.rolling(wavelength=5).mean() for d in sel]								# smooth along wavelength
# sel = [d.interp(angle=np.linspace(d.angle.min(), d.angle.max(), 200)) for d in sel]# interpolate angle
pl = sel[0] + sel[1]
pol = (sel[0] - sel[1])/pl

plt.figure(figsize=FIGSIZE_WIDE)
plt.subplot(1,2,1)
pl.plot(cmap=CMAP)
plt.subplot(1,2,2)
pol.plot(cmap=CMAP)
plt.ylabel("")
# plot.energy_ticks()
plt.show()

['2023-12-01/d004_pol_30K_0T_647nm_rotatingPolDetektor_30degLaserPolarisation.hd5', '2023-12-01/d005_pol_30K_0T_647nm_rotatingPolDetektor_72degLaserPolarisation.hd5']


## systematic excitation and detection polarisation dependence measurement

In [24]:
def _fit_sin_sqr(x, *p):
	return p[0] * np.sin(x + p[1])**2 + p[2]

def fit_sin_sqr(intens, angle):
	popt, pcov = sp.optimize.curve_fit(_fit_sin_sqr, angle, intens, p0=[1, 0, 0])
	angle = np.linspace(0, 2*np.pi, 100)
	plt.plot(angle, _fit_sin_sqr(angle, *popt))
	return popt, pcov

In [25]:
# systematic excitation and detection polarisation dependence
sel = [
	d for d in data if d.name.startswith("2023-12-04") and "d002" in d.name
]
sel = [d.copy() for d in sel]
sel = [subtract_background(d) for d in sel]
sel = [d.rolling(wavelength=5).mean() for d in sel]								# smooth along wavelength

print(sel[0].name)

def extract_PolDet(p): 
	return float(p.split("degPolDet")[0].split("_")[-1])

def norm(d):
	return d.max("wavelength")

plt.figure(figsize=FIGSIZE_WIDE)

plt.subplot(1,2,1)
img = np.array([d.max("wavelength").data for d in sel])
plt.imshow(img, extent=[0, 180, 0, 90], origin="lower", 
	interpolation="bicubic"
)
plt.xlabel("excite pol. angle")
plt.ylabel("det. pol. angle")
plt.colorbar()

def fit(x, *p): return p[0] * np.cos(x + p[1])**2 + p[2]

popt = []
popt_unc = []
ax = plt.subplot(1,2,2, projection="polar")
for d in sel:
	d = d.assign_coords(angle=d.angle * np.pi/180)
	d = d.assign_coords(angle=d.angle * 2)
	norm(
		d
	).plot.scatter(label=f"det. {extract_PolDet(d.name)*2}°", s=30)
	opot, pcov = sp.optimize.curve_fit(fit, d.angle, norm(d), p0=[1, 0, 0])
	angle = np.linspace(0, 2*np.pi, 100)
	plt.plot(angle, fit(angle, *opot))
	popt.append(opot)
	popt_unc.append(np.sqrt(np.diag(pcov)))
plt.legend(bbox_to_anchor=(1.1, 1))
plt.xlabel("excite pol. angle")
plt.title("")
plt.ylabel("")
ax.set_rlabel_position(-90)

plt.show()

print(f"best excitation axis:\t{(np.abs(np.array(popt)[:,1])%(np.pi/2)).mean() * 180/np.pi: .2f}° +- {(np.abs(np.array(popt_unc)[:,1])%(np.pi/2)).mean() * 180/np.pi: .2f}°")

2023-12-04_LO_MG_NiPS3/d002_pol_30K_0T_647nm_0degPolDet_rotatingPolExc
best excitation axis:	 51.45° +-  0.73°


In [26]:
plt.figure()
ax = plt.subplot(1,1,1, projection="polar")

angle = [2*extract_PolDet(d.name) * np.pi/180 for d in sel]
ampl =  np.abs(np.array(popt)[:,2]) + np.abs(np.array(popt)[:,0])

plt.plot(
	[2*extract_PolDet(d.name) * np.pi/180 for d in sel],
	np.abs(np.array(popt)[:,2]) + np.abs(np.array(popt)[:,0]),
	"o"
)
popt_det, pcov_det = fit_sin_sqr(ampl, angle)
angles = np.linspace(0, 2*np.pi, 100)
plt.plot(angles, _fit_sin_sqr(angles, *popt_det))

print(f"axis:\t{(popt_det[1]%(np.pi/2)) * 180/np.pi: .2f}° +- {(np.sqrt(np.diag(pcov_det))[1]%(np.pi/2)) * 180/np.pi: .2f}°")

plt.xlabel("detection polarisation angle")
plt.show()

axis:	 51.25° +-  11.01°


## PL pol. over magnetic field

In [27]:
sel = data[21:24]

def extract_field(p):
	return float(p.split("T")[0].split("_")[-1])

def norm(d):
	return d.max("wavelength")

sel = [subtract_background(d) for d in sel]

popt, pcov = [], []
for d in sel:
	d = d.assign_coords(angle=d.angle * np.pi/180)
	d = d.assign_coords(angle=d.angle * 2)
	i_pot, i_cov = sp.optimize.curve_fit(
		 _fit_sin_sqr, d.angle, norm(d), p0=[0, 0, 0],
	)
	angle = np.linspace(0, 2*np.pi, 100)
	popt.append(i_pot)
	pcov.append(i_cov)

field = [extract_field(d.name) for d in sel]

plt.figure(figsize=FIGSIZE_WIDE)
plt.subplot(1,2,1)
plt.errorbar(
	field,
	# np.abs(np.array(popt)[:,1])%(np.pi/2) * 180/np.pi,
	np.abs(np.array(popt)[:,1])%(np.pi/2) * 180/np.pi,
	yerr=np.sqrt(np.diag(np.array(pcov)[:,1]))%(np.pi/2) * 180/np.pi,
	marker="o",
	linestyle="none"
)
plt.xlabel("field [T]")
plt.ylabel("excitation axis [°]")

ax = plt.subplot(1,2,2,
	# projection="polar"
)

axis = []
for f, d, p in zip(field, sel, popt):
	d = d.assign_coords(angle=d.angle * np.pi/180)
	d = d.assign_coords(angle=d.angle * 2)
	norm(
		d
	).plot.scatter(label=f"{f} T", 
		s=30
	)
	angle = np.linspace(0, np.pi, 100)
	plt.plot(angle, _fit_sin_sqr(angle, *p))
# ax.set_rlabel_position(-90)
plt.legend(
	# bbox_to_anchor=(1.1, 1)
)
plt.ylabel("")

plt.show()